In [72]:
# !pip install presidio_analyzer
# !pip install presidio-anonymizer
# !pip install names
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngineProvider, TransformersNlpEngine, NerModelConfiguration
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import RecognizerResult, OperatorConfig
import names
import random
from random import randrange
from datetime import timedelta, datetime
import spacy
import json
import re

# Create configuration containing engine name and models
configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "es", "model_name": "es_core_news_md"},
                {"lang_code": "en", "model_name": "en_core_web_lg"}],
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_with_spanish = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine_with_spanish, 
    supported_languages=["en", "es"]
)

In [73]:
# text_transcripts = ["""
# **Tutor:** "Hello, Alex! I'm dAvid, your math tutor. Today we're going to work on fractions, specifically adding and subtracting them. Are you ready?"

# **Student (Alex):** "Yes, I think so. Fractions are a bit confusing, though."

# **Tutor:** "No worries, we'll tackle them step by step. Let's start with adding fractions. Do you remember what we need to do first?"

# **Alex:** "We need to make sure the denominators are the same, right?"

# **Tutor:** "Exactly, Alex! That's very important. If we have 1/4 + 3/4, what would be the sum?"

# **Alex:** "Since the denominators are the same, it would be 4/4, which is the same as one whole, right?"

# **Tutor:** "Spot on! Now, what if we have different denominators, like 1/2 + 1/3?"

# **Alex:** "We need to find a common denominator, right? But I'm not sure how to do that."

# **Tutor:** "Correct, we need a common denominator. For 1/2 and 1/3, the smallest number that both 2 and 3 can divide into evenly is 6. So, we convert the fractions to have the denominator of 6. Can you try that?"

# **Alex:** "So, 1/2 becomes 3/6, and 1/3 becomes 2/6?"

# **Tutor:** "Exactly, well done! Now, what's 3/6 + 2/6?"

# **Alex:** "Is it 5/6?"

# **Tutor:** "Perfect! You're getting the hang of this. Let's try some subtraction next. Ready?"

# **Alex:** "Yes, let's do it!"
# """]

In [74]:
# text_transcripts = [
#     """
#     **Tutor:** "Hello, Alex! I'm Jionghao Lin, your math tutor. Today we're going to work on fractions, specifically adding and subtracting them. Are you ready?"

#     **Student (Alex):** "Yes, I think so. Fractions are a bit confusing, though."

#     **Tutor:** "No worries, we'll tackle them step by step. Let's start with adding fractions. Do you remember what we need to do first?"

#     **Alex:** "We need to make sure the denominators are the same, right?"

#     **Tutor:** "Exactly, Alex! That's very important. If we have 1/4 + 3/4, what would be the sum?"

#     **Alex:** "Since the denominators are the same, it would be 4/4, which is the same as one whole, right?"

#     **Tutor:** "Spot on! Now, what if we have different denominators, like 1/2 + 1/3?"

#     **Alex:** "We need to find a common denominator, right? But I'm not sure how to do that."

#     **Tutor:** "Correct, we need a common denominator. For 1/2 and 1/3, the smallest number that both 2 and 3 can divide into evenly is 6. So, we convert the fractions to have the denominator of 6. Can you try that?"

#     **Alex:** "So, 1/2 becomes 3/6, and 1/3 becomes 2/6?"

#     **Tutor:** "Exactly, well done! Now, what's 3/6 + 2/6?"

#     **Alex:** "Is it 5/6?"

#     **Tutor:** "Perfect! You're getting the hang of this. Let's try some subtraction next. Ready?"

#     **Alex:** "Yes, let's do it!"
#     """,
#     """
#     **Tutor:** "Hi there, Emily! My name is Mark Davis, and I'll be helping you with your science project today. Are you excited?"

#     **Student (Emily):** "Hi Mark! Yes, I'm very excited. I love science!"

#     **Tutor:** "That's great to hear! Let's start by discussing your project's topic. I see you've chosen photosynthesis. Is that correct?"

#     **Emily:** "Yes, I find it fascinating how plants make their food."

#     **Tutor:** "It's a wonderful topic. We'll also review your project due date, which is September 25, 2024. Does that sound good?"

#     **Emily:** "Yes, that works for me."
#     """,
#     """
#     **Tutor:** "Good afternoon, Jake! I'm Sarah Wilson, your history tutor. Today, we'll focus on the American Civil War. Do you have your notes with you?"

#     **Student (Jake):** "Hi Sarah! Yes, I have them here. I'm ready to learn."

#     **Tutor:** "Fantastic! Before we dive in, I noticed your project presentation is on November 14, 2024. Is that still the plan?"

#     **Jake:** "Yes, that's correct."

#     **Tutor:** "Great. Let's get started with the main causes of the Civil War. We'll also need to verify some dates. Can you give me a call at (555) 123-4567 if you have any questions later?"

#     **Jake:** "Sure, I'll call if I need any help."
#     """,
#     """
#     **Tutor:** "Hello, Lily! My name is John Brown, and I'm here to help you with your English literature homework. Are you ready to get started?"

#     **Student (Lily):** "Hi John! Yes, I'm ready."

#     **Tutor:** "Excellent. We'll focus on Shakespeare today. Before that, can you confirm your email address? I have it as lily.smith@example.com. Is that correct?"

#     **Lily:** "Yes, that's my email."

#     **Tutor:** "Perfect. Let's start with 'Romeo and Juliet.'"
#     """,
#     """
#     **Tutor:** "Good morning, Noah! I'm Jessica Lee, your geography tutor. Today, we'll learn about the continents. Are you prepared?"

#     **Student (Noah):** "Hi Jessica! Yes, I am."

#     **Tutor:** "Great! Also, if you need any additional help, you can email me at jessica.lee@tutoring.com. Now, can you tell me which continent is the largest?"

#     **Noah:** "I think it's Asia, right?"

#     **Tutor:** "Correct! Let's explore more about Asia."
#     """,
#     """
#     **Tutor:** "Hi, Ava! I'm Michael Green, and I'll be tutoring you in math today. Are you ready to dive into algebra?"

#     **Student (Ava):** "Hi Michael! Yes, I'm ready."

#     **Tutor:** "Fantastic. I also wanted to let you know that our next session is scheduled for October 10, 2024. Does that work for you?"

#     **Ava:** "Yes, that's perfect."

#     **Tutor:** "Great. Let's start with solving equations. What's the value of x in 2x + 3 = 7?"

#     **Ava:** "x is 2, right?"

#     **Tutor:** "Exactly! Well done."
#     """,
#     """
#     **Tutor:** "Hello, Mia! My name is David Johnson, and I'll be your biology tutor today. Are you excited?"

#     **Student (Mia):** "Hi David! Yes, I love biology."

#     **Tutor:** "Wonderful! I see your project is due on December 5, 2024. Is that correct?"

#     **Mia:** "Yes, that's right."

#     **Tutor:** "Great. If you need to discuss anything, you can call me at (555) 987-6543. Now, let's talk about cell structures. Can you name the parts of a cell?"

#     **Mia:** "Sure, there's the nucleus, mitochondria, and ribosomes."

#     **Tutor:** "Excellent! Let's dive deeper into each part."
#     """,
#     """
#     **Tutor:** "Hi, Sophia! I'm Emma Martinez, and I'll be helping you with your history assignment today. Are you ready to begin?"

#     **Student (Sophia):** "Hi Emma! Yes, I'm ready."

#     **Tutor:** "Great. I noticed your project due date is January 15, 2025. Is that still accurate?"

#     **Sophia:** "Yes, it is."

#     **Tutor:** "Perfect. Also, if you have any questions, feel free to email me at emma.martinez@tutoringhelp.com. Let's start with the ancient civilizations. Which one do you want to learn about first?"

#     **Sophia:** "I'd like to learn about Ancient Egypt."

#     **Tutor:** "Great choice! Let's get started."
#     """,
#     """
#     **Tutor:** "Good afternoon, Lucas! My name is Olivia Davis, and I'm here to help you with your physics homework. How are you today?"

#     **Student (Lucas):** "Hi Olivia! I'm good, thank you."

#     **Tutor:** "That's wonderful. Your homework is due on October 18, 2024. Does that sound right?"

#     **Lucas:** "Yes, that's correct."

#     **Tutor:** "Perfect. If you need any further assistance, you can call me at (555) 321-4321. Now, let's discuss Newton's laws of motion. Can you name the first law?"

#     **Lucas:** "Is it the law of inertia?"

#     **Tutor:** "Exactly! Let's explore it further."
#     """,
#     """
#     **Tutor:** "Hello, Emma! My name is Robert Wilson, and I'll be assisting you with your chemistry assignment today. Are you ready?"

#     **Student (Emma):** "Hi Robert! Yes, I'm ready."

#     **Tutor:** "Excellent. I see your assignment is due on November 22, 2024. Is that correct?"

#     **Emma:** "Yes, that's right."

#     **Tutor:** "Great. If you need to reach me, my email is robert.wilson@tutors.com. Let's start with the periodic table. Can you tell me the symbol for sodium?"

#     **Emma:** "It's Na."

#     **Tutor:** "Perfect! Let's learn more about its properties."
#     """,
#     """
#     **Tutor:** "Hi, Daniel! I'm Laura Kim, your new math tutor. Are you ready to work on some geometry today?"

#     **Student (Daniel):** "Hi Laura! Yes, I'm ready."

#     **Tutor:** "Fantastic. Your test is coming up on December 12, 2024. Is that date still correct?"

#     **Daniel:** "Yes, it is."

#     **Tutor:** "Great. If you have any questions, you can reach me at laura.kim@mathhelp.com. Let's start with triangles. Can you tell me the different types of triangles?"

#     **Daniel:** "Sure, there are equilateral, isosceles, and scalene triangles."

#     **Tutor:** "Exactly! Let's explore each type in detail."
#     """
# ]

In [75]:
# Load the processed_transcripts here
with open('processed_transcripts.txt', 'r') as f:
    text_transcripts = json.load(f)

# Verify the content
print(len(text_transcripts))

260


In [76]:
# Updated code: change tutor's and student's names to different fake names.

def de_identify_pii(text_transcript):
    # Initialize the analyzer and anonymizer
    analyzer = AnalyzerEngine()
    anonymizer = AnonymizerEngine()

    # Define date range for generating random dates and generate a random date
    d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
    d2 = datetime.strptime('1/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')
    random_date = (d1 + timedelta(days=random.randint(0, (d2 - d1).days))).strftime('%m/%d/%Y')

    # Function to generate a unique fake name
    def generate_fake_name(existing_names):
        while True:
            fake_name = names.get_first_name()
            if fake_name not in existing_names:
                return fake_name

    # Analyze the text to find PII
    results_english = analyzer.analyze(text=text_transcript, language="en", return_decision_process=True)
    # <PERSON>, <DATE_TIME>, ...
    
    # Create a mapping of original names to unique fake names
    # Is creating a mapping good idea? User can easily print out the name_mapping dictionary and see the PII...
    # So we cannot release code...
    name_mapping = {}
    existing_names = set()
    for result in results_english:
        if result.entity_type == "PERSON":
            original_name = text_transcript[result.start:result.end]
            if original_name not in name_mapping:
                fake_name = generate_fake_name(existing_names)
                name_mapping[original_name] = fake_name
                existing_names.add(fake_name)

    # print(name_mapping)

    operators = {
        "PERSON": OperatorConfig("custom", {"lambda": lambda text : name_mapping.get(text, text)}),
        "DATE_TIME": OperatorConfig("replace", {"new_value": random_date})
        # Add more categories
        # "EMAIL_ADDRESS":
        # "LOCATION":
        # "PHONE_NUMBER":
    }

    # Anonymize the text
    result = anonymizer.anonymize(
        text=text_transcript,
        analyzer_results=results_english,
        operators=operators
    )

    return result, results_english


In [77]:
# def highlight_text(text, entities):
#     highlighted_text = ""
#     last_end = 0
#     for entity in entities:
#         start, end = entity['start'], entity['end']
#         # Add the text up to the current entity
#         highlighted_text += text[last_end:start]
#         # Highlight the current entity
#         highlighted_text += '\x1b[6;30;42m'+ f"[{text[start:end]}]" +'\x1b[0m'
#         last_end = end
#     # Add any remaining text after the last entity
#     highlighted_text += text[last_end:]
#     return highlighted_text

def highlight_text(text, entities):
    highlighted_text = ""
    last_end = 0
    
    # Sort entities by their start position
    entities = sorted(entities, key=lambda x: x['start'])
    
    for entity in entities:
        start, end = entity['start'], entity['end']
        if start >= last_end:  # Ensure there's no overlap
            # Add the text up to the current entity
            highlighted_text += text[last_end:start]
            # Highlight the current entity
            highlighted_text += '\x1b[6;30;42m' + f"[{text[start:end]}]" + '\x1b[0m'
            last_end = end
    
    # Add any remaining text after the last entity
    highlighted_text += text[last_end:]
    return highlighted_text

def highlight_preserving_format(text, entities):
    # Highlight the text
    highlighted_text = highlight_text(text, entities)
    
    # Split the highlighted text into lines to preserve original formatting
    highlighted_lines = highlighted_text.split('\n')
    
    # Join the lines back into a single string with newline characters
    return '\n'.join(highlighted_lines)

def highlight_orig_text(text_transcript, results_english):
    entities_info = []

    for res in results_english:
        entities_info.append({'type':res.entity_type, 'start':res.start, 'end':res.end})

    # Ensure entities are sorted by their start position
    entities_info.sort(key=lambda x: x["start"])

    # Highlight the text
    highlighted_orig = highlight_preserving_format(text_transcript, entities_info)

    # Printing the highlighted text
    return highlighted_orig

def hightlight_replaced_text(result):
    results_english_replaced = analyzer.analyze(text=result.text, language="en", 
                        return_decision_process=True)

    entities_info_replaced = []

    for res in results_english_replaced:
        # print(res)
        entities_info_replaced.append({'type':res.entity_type, 'start':res.start, 'end':res.end})

    # Ensure entities are sorted by their start position
    entities_info_replaced.sort(key=lambda x: x["start"])

    # Highlight the text
    highlighted_replaced = highlight_preserving_format(result.text, entities_info_replaced)

    highlighted_replaced = re.sub(r'(<[^>]+>)', r'\033[42;30m\1\033[0m', highlighted_replaced)

    # Printing the highlighted text
    return highlighted_replaced

In [78]:
def run_all(text_transcripts):
    for text_transcript in text_transcripts[:10]:
        result, results_english = de_identify_pii(text_transcript)
        highlighted_orig = highlight_orig_text(text_transcript, results_english)
        print(highlighted_orig)
        print("----------------------------------------------------------------------------------------------------------------------------")
        highlighted_replaced = hightlight_replaced_text(result)
        print(highlighted_replaced)
        print("----------------------------------------------------------------------------------------------------------------------------")
        print("----------------------------------------------------------------------------------------------------------------------------")

run_all(text_transcripts)

teacher: Hi [Emma Johnson], hope I didn't get you up too early!
student: Don't worry, my exam is [next Saturday], so I should get up early.
teacher: Ah OK, so good practice [then...is] that an IELTS exam?
student: Exactly.
teacher: I've lost track of how many you've done
student: I hope I can do as well as usual at the real exam.haha
teacher: Do you mean get a score you get in practice tests?
student: Yes, I managed to get 37 on reading [yesterday]!!
teacher: Wow that's really good - well done...you've definitely made a lot of progress. And what's your normal score on listening [these days]?
student: Around 30, the listening is harder for me.
teacher: OK that's quite common to be better at reading - but 30 is still more than respectable. I guess maybe speaking is the toughest part?
student: Yeah, because it's difficult to practice...
teacher: Yeah sure - I guess what we're doing right now is a weird mixture of speaking and [writing....be]cause you're kind of under pressure to reply str